In [1]:
import os

os.chdir("../..")

In [2]:
!ls results/

AA_generated_results.csv  AV_generated_results.csv  model_output_summary.csv
AA_original_results.csv   AV_original_results.csv


In [26]:
import pandas as pd


model_renamer = {
    'gemma-3-27b-it': "Gemma-3-27B", 
    'gpt-4o-mini-2024-07-18': "GPT-4o-mini", 
    'gemini-2.0-flash': "Gemini-2.0-Flash", 
    'gpt-4o-2024-08-06': "GPT-4o", 
    'deepseek-chat': "DeepSeek-V3",
    'llama-4-maverick-17b-128e-instruct': "Llama-4-Maverick",
}

dataset_renamer = {
    "enron": "Enron",
    "CCAT50": "CCAT50",
    "blog": "Blog",
    "reddit": "Reddit",
}

# setting_renamer = {
#     "Setting1": "5-shot",
#     "Setting4": "0-shot",}

df = pd.read_csv("results/AV_generated_results.csv")
df.LLM_Model = df.LLM_Model.map(model_renamer)
df.Dataset = df.Dataset.map(dataset_renamer)
# df.Setting = df.Setting.map(setting_renamer)
df = df.copy()[df.Setting.isin(["Setting1", "Setting4"])].reset_index(drop=True)
df

,Setting,Dataset,LLM_Model,AV_Model,Top_1
0,Setting4,Enron,Gemma-3-27B,longformer-base-4096,84.09
1,Setting4,Enron,Gemma-3-27B,ModernBERT-base,85.87
2,Setting4,Enron,GPT-4o-mini,longformer-base-4096,84.65
3,Setting4,Enron,GPT-4o-mini,ModernBERT-base,86.71
4,Setting4,Enron,Gemini-2.0-Flash,longformer-base-4096,85.22
...,...,...,...,...,...
75,Setting1,CCAT50,Gemini-2.0-Flash,ModernBERT-base,98.76
76,Setting1,CCAT50,Llama-4-Maverick,longformer-base-4096,93.72
77,Setting1,CCAT50,Llama-4-Maverick,ModernBERT-base,95.64
78,Setting1,CCAT50,GPT-4o,longformer-base-4096,93.44


In [27]:
df.LLM_Model.unique()

array(['Gemma-3-27B', 'GPT-4o-mini', 'Gemini-2.0-Flash', 'GPT-4o',
       'DeepSeek-V3', 'Llama-4-Maverick'], dtype=object)

In [30]:
cols = ['Dataset', 'Setting', 'GPT-4o', 'GPT-4o-mini', 
        'Gemini-2.0-Flash', 'Gemma-3-27B', 'DeepSeek-V3', 'Llama-4-Maverick']

pvt = df.pivot_table(
    index=["Dataset", "Setting"],
    columns=["LLM_Model"],
    values="Top_1",
    aggfunc="mean",
).reset_index().fillna("-")[cols]
pvt

LLM_Model,Dataset,Setting,GPT-4o,GPT-4o-mini,Gemini-2.0-Flash,Gemma-3-27B,DeepSeek-V3,Llama-4-Maverick
0,Blog,Setting1,19.365,17.935,21.250,16.720,20.775,17.61
1,Blog,Setting4,8.150,8.220,10.350,9.555,-,-
2,CCAT50,Setting1,95.280,95.160,97.440,97.340,97.46,94.68
3,CCAT50,Setting4,93.140,93.380,93.940,94.180,-,-
4,Enron,Setting1,96.150,96.640,96.435,96.170,96.295,95.65
5,Enron,Setting4,85.015,85.680,85.955,84.980,-,-
6,Reddit,Setting1,63.650,60.225,65.880,55.535,49.97,65.1
7,Reddit,Setting4,56.660,54.285,53.255,48.750,-,-


In [32]:
print(pvt.to_latex(index=False, escape=False, float_format="%.2f",
    multirow=True, multicolumn=True, column_format="llcccccc",))

\begin{tabular}{llcccccc}
\toprule
Dataset & Setting & GPT-4o & GPT-4o-mini & Gemini-2.0-Flash & Gemma-3-27B & DeepSeek-V3 & Llama-4-Maverick \\
\midrule
Blog & Setting1 & 19.37 & 17.93 & 21.25 & 16.72 & 20.77 & 17.61 \\
Blog & Setting4 & 8.15 & 8.22 & 10.35 & 9.55 & - & - \\
CCAT50 & Setting1 & 95.28 & 95.16 & 97.44 & 97.34 & 97.46 & 94.68 \\
CCAT50 & Setting4 & 93.14 & 93.38 & 93.94 & 94.18 & - & - \\
Enron & Setting1 & 96.15 & 96.64 & 96.44 & 96.17 & 96.30 & 95.65 \\
Enron & Setting4 & 85.02 & 85.68 & 85.95 & 84.98 & - & - \\
Reddit & Setting1 & 63.65 & 60.23 & 65.88 & 55.54 & 49.97 & 65.10 \\
Reddit & Setting4 & 56.66 & 54.28 & 53.25 & 48.75 & - & - \\
\bottomrule
\end{tabular}

